In [ ]:
# getting mocap data
from support.funcs import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df_mocap,starttime=read_df_csv(r'C:\Users\arpan\OneDrive\Documents\internship\mocapcsv\arpan_t2.csv')

# converting to epoch time
epoch_time = starttime.timestamp() 
time=df_mocap['seconds'].to_numpy()
time=time+epoch_time
df_mocap.insert(1,'epoch_time',time)
df_mocap.columns

# getting mpipe data
df_mpipe=pd.read_csv('mpipe.csv')
df_mpipe

# converting mpipe to mocap frame
rotmat=[]
org=[]
with open(r'D435_rotmat.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        x=x.replace(']','')
        x=x.replace('[','')
        line=x.split(' ')
        while ' ' in line:
            line=line.remove(' ')
        #k_rotmat.append([float(x)])
        while '' in line:
            ind=line.index('')
            line.pop(ind)
        x=[]
        for i in line:
            x.append(float(i))
        rotmat.append(x)
    rotmat=np.array(rotmat)

with open(r'D435_org.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        x=x.replace(']','')
        x=x.replace('[','')
        org.append([float(x)])
    k_org=np.array(org)

# print(rotmat)
# print(org)

for index,j in df_mpipe.iterrows():
    for k in range(1,1+7*3,3):
        point=[]
        for p in range(k,k+3):
            point.append(j[p])
        converted_point=frame_con(point,rotmat,org)
        # print(converted_point)
        for o in range(3):
            df_mpipe.iloc[index,k+o]=converted_point[o]

df_mpipe

# occlusion based on limb length
df_limb=pd.read_csv('limbl.csv')
oLul,oLll,oLf,oRul,oRll,oRf,oH=[0],[0],[0],[0],[0],[0],[0]
th=0.10 # 10cm
for i in range(1,len(df_limb)):
    if abs(df_limb['Lul'][i]-df_limb['Lul'].mean())>th:
        oLul.append(1)
    else:
        oLul.append(0)
    if abs(df_limb['Lll'][i]-df_limb['Lll'].mean())>th:
        oLll.append(1)
    else:
        oLll.append(0)
    if abs(df_limb['Lf'][i]-df_limb['Lf'].mean())>th:
        oLf.append(1)
    else:
        oLf.append(0)
    if abs(df_limb['Rul'][i]-df_limb['Rul'].mean())>th:
        oRul.append(1)
    else:
        oRul.append(0)
    if abs(df_limb['Rll'][i]-df_limb['Rll'].mean())>th:
        oRll.append(1)
    else:
        oRll.append(0)
    if abs(df_limb['Rf'][i]-df_limb['Rf'].mean())>th:
        oRf.append(1)
    else:
        oRf.append(0)
    if abs(df_limb['H'][i]-df_limb['H'].mean())>th:
        oH.append(1)
    else:
        oH.append(0)
    
# filtering occlusion limb length
for index,j in df_mpipe.iterrows():
    
    for k in range(3):
        if oLul[index]==0 and oH[index]==0:
            LHx=df_mpipe['LH_x'].iloc[index]
            LHy=df_mpipe['LH_y'].iloc[index]
            LHz=df_mpipe['LH_z'].iloc[index]
        df_mpipe['LH_x'].iloc[index]=LHx
        df_mpipe['LH_y'].iloc[index]=LHy
        df_mpipe['LH_z'].iloc[index]=LHz
        
    for k in range(3):
        if oLll[index]==0 and oLul[index]==0:
            LKx=df_mpipe['LK_x'].iloc[index]
            LKy=df_mpipe['LK_y'].iloc[index]
            LKz=df_mpipe['LK_z'].iloc[index]
        df_mpipe['LK_x'].iloc[index]=LKx
        df_mpipe['LK_y'].iloc[index]=LKy
        df_mpipe['LK_z'].iloc[index]=LKz

    for k in range(3):
        if oLll[index]==0 and oLf[index]==0:
            LAx=df_mpipe['LA_x'].iloc[index]
            LAy=df_mpipe['LA_y'].iloc[index]
            LAz=df_mpipe['LA_z'].iloc[index]
        df_mpipe['LA_x'].iloc[index]=LAx
        df_mpipe['LA_y'].iloc[index]=LAy
        df_mpipe['LA_z'].iloc[index]=LAz

    for k in range(3):
        if oLf[index]==0:
            LFx=df_mpipe['LT_x'].iloc[index]
            LFy=df_mpipe['LT_y'].iloc[index]
            LFz=df_mpipe['LT_z'].iloc[index]
        df_mpipe['LT_x'].iloc[index]=LFx
        df_mpipe['LT_y'].iloc[index]=LFy
        df_mpipe['LT_z'].iloc[index]=LFz

    for k in range(3):
            if oRul[index]==0 and oH[index]==0:
                RHx=df_mpipe['RH_x'].iloc[index]
                RHy=df_mpipe['RH_y'].iloc[index]
                RHz=df_mpipe['RH_z'].iloc[index]
            df_mpipe['RH_x'].iloc[index]=RHx
            df_mpipe['RH_y'].iloc[index]=RHy
            df_mpipe['RH_z'].iloc[index]=RHz
        
    for k in range(3):
        if oRll[index]==0 and oRul[index]==0:
            RKx=df_mpipe['RK_x'].iloc[index]
            RKy=df_mpipe['RK_y'].iloc[index]
            RKz=df_mpipe['RK_z'].iloc[index]
        df_mpipe['RK_x'].iloc[index]=RKx
        df_mpipe['RK_y'].iloc[index]=RKy
        df_mpipe['RK_z'].iloc[index]=RKz


    for k in range(3):
        if oRll[index]==0 and oRf[index]==0:
            RAx=df_mpipe['RA_x'].iloc[index]
            RAy=df_mpipe['RA_y'].iloc[index]
            RAz=df_mpipe['RA_z'].iloc[index]
        df_mpipe['RA_x'].iloc[index]=RAx
        df_mpipe['RA_y'].iloc[index]=RAy
        df_mpipe['RA_z'].iloc[index]=RAz


    for k in range(3):
        if oRf[index]==0:
            RFx=df_mpipe['RT_x'].iloc[index]
            RFy=df_mpipe['RT_y'].iloc[index]
            RFz=df_mpipe['RT_z'].iloc[index]
        df_mpipe['RT_x'].iloc[index]=RFx
        df_mpipe['RT_y'].iloc[index]=RFy
        df_mpipe['RT_z'].iloc[index]=RFz

def pvt(x,t):
    from scipy.signal import savgol_filter
    y=[]
    for i in x:
        y.append(i)
    # y=[i for i in y if str(i) != 'nan']
    w = savgol_filter(y,  20,2)
    plt.plot(t,w)
    # plt.ylim(-1,1)
    plt.xlabel('time(s)')

In [ ]:
# calibration frame get
f=False
for ind,val in enumerate(df_mpipe['epoch_time'].tolist()):
    for i,v in enumerate(df_mocap['epoch_time'].tolist()):
        if v==val:
            print(ind,i,val)
            mpipe_cf=ind
            mocap_cf=i
            f=True
            break
    if f:
        break

print('\n-----------------------\n')

for ind,val in enumerate(df_mpipe['epoch_time'].tolist()):
    for i,v in enumerate(df_mocap['epoch_time'].tolist()):
        if v==val:
            print(ind,i,val)

In [ ]:
#calculating the knee angle
mpipeRKa, mocapRKa=[],[]

for i in range(len(df_mpipe['epoch_time'].tolist())):
    mpipeRKa.append(angle3point([df_mpipe['RH_x'][i],df_mpipe['RH_y'][i],df_mpipe['RH_z'][i]],
                                [df_mpipe['RK_x'][i],df_mpipe['RK_y'][i],df_mpipe['RK_z'][i]],
                                [df_mpipe['RA_x'][i],df_mpipe['RA_y'][i],df_mpipe['RA_z'][i]]))
   
# for i in range(len(df_mocap['epoch_time'].tolist())):
#     mocapRKa.append(angle3point([df_mocap['RH_x'][i],df_mocap['RH_y'][i],df_mocap['RH_z'][i]],
#                                 [df_mocap['RK_x'][i],df_mocap['RK_y'][i],df_mocap['RK_z'][i]],
#                                 [df_mocap['RA_x'][i],df_mocap['RA_y'][i],df_mocap['RA_z'][i]]))
    
#test graph right elbow angle wihtout occlusion
pvt(mpipeRKa,df_mpipe['epoch_time'])
# pvt(mocapRKa,df_mocap['epoch_time'])
plt.title('Right knee Angle in degrees ')
plt.xlabel('time(s)')
plt.ylabel('Angle(degrees)')
# plt.legend(['mediapipe','mocap'])
plt.ylim(0,180)
plt.show()

# print('RMSE of right knee Angle:',RMSE(mocapRKa,mpipeRKa,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))

mpipeLKa, mocapLKa=[],[]

for i in range(len(df_mpipe['epoch_time'].tolist())):
    mpipeLKa.append(angle3point([df_mpipe['LH_x'][i],df_mpipe['LH_y'][i],df_mpipe['LH_z'][i]],
                                [df_mpipe['LK_x'][i],df_mpipe['LK_y'][i],df_mpipe['LK_z'][i]],
                                [df_mpipe['LA_x'][i],df_mpipe['LA_y'][i],df_mpipe['LA_z'][i]]))
   
# for i in range(len(df_mocap['epoch_time'].tolist())):
#     mocapLKa.append(angle3point([df_mocap['LH_x'][i],df_mocap['LH_y'][i],df_mocap['LH_z'][i]],
#                                 [df_mocap['LK_x'][i],df_mocap['LK_y'][i],df_mocap['LK_z'][i]],
#                                 [df_mocap['LA_x'][i],df_mocap['LA_y'][i],df_mocap['LA_z'][i]]))
   
#test graph right elbow angle wihtout occlusion
pvt(mpipeLKa,df_mpipe['epoch_time'])
# pvt(mocapLKa,df_mocap['epoch_time'])
plt.title('Left knee Angle in degrees ')
plt.xlabel('time(s)')
plt.ylabel('Angle(degrees)')
# plt.legend(['mediapipe','mocap'])
plt.ylim(0,180)
plt.show()
# print('RMSE of left knee Angle:',RMSE(mocapLKa,mpipeLKa,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))


In [ ]:
#calculating the ankle angle
mpipeRAa, mocapRaa=[],[]

for i in range(len(df_mpipe['epoch_time'].tolist())):
    mpipeRAa.append(angle3point([df_mpipe['RT_x'][i],df_mpipe['RT_y'][i],df_mpipe['RT_z'][i]],
                                [df_mpipe['RA_x'][i],df_mpipe['RA_y'][i],df_mpipe['RA_z'][i]],
                                [df_mpipe['RK_x'][i],df_mpipe['RK_y'][i],df_mpipe['RK_z'][i]]))
   
# for i in range(len(df_mocap['epoch_time'].tolist())):
#     mocapRAa.append(angle3point([df_mocap['RT_x'][i],df_mocap['RT_y'][i],df_mocap['RT_z'][i]],
#                                 [df_mocap['RK_x'][i],df_mocap['RK_y'][i],df_mocap['RK_z'][i]],
#                                 [df_mocap['RA_x'][i],df_mocap['RA_y'][i],df_mocap['RA_z'][i]]))
    
#test graph right elbow angle wihtout occlusion
pvt(mpipeRAa,df_mpipe['epoch_time'])
# pvt(mocapRAa,df_mocap['epoch_time'])
plt.title('Right ankle Angle in degrees ')
plt.xlabel('time(s)')
plt.ylabel('Angle(degrees)')
# plt.legend(['mediapipe','mocap'])
plt.ylim(0,180)
plt.show()

# print('RMSE of right knee Angle:',RMSE(mocapRKa,mpipeRKa,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))

mpipeLAa, mocapLAa=[],[]

for i in range(len(df_mpipe['epoch_time'].tolist())):
    mpipeLAa.append(angle3point([df_mpipe['LT_x'][i],df_mpipe['LT_y'][i],df_mpipe['LT_z'][i]],
                                [df_mpipe['LA_x'][i],df_mpipe['LA_y'][i],df_mpipe['LA_z'][i]],
                                [df_mpipe['LK_x'][i],df_mpipe['LK_y'][i],df_mpipe['LK_z'][i]]))
   
# for i in range(len(df_mocap['epoch_time'].tolist())):
#     mocapLAa.append(angle3point([df_mocap['LT_x'][i],df_mocap['LT_y'][i],df_mocap['LT_z'][i]],
#                                 [df_mocap['LK_x'][i],df_mocap['LK_y'][i],df_mocap['LK_z'][i]],
#                                 [df_mocap['LA_x'][i],df_mocap['LA_y'][i],df_mocap['LA_z'][i]]))
   
#test graph right elbow angle wihtout occlusion
pvt(mpipeLAa,df_mpipe['epoch_time'])
# pvt(mocapLAa,df_mocap['epoch_time'])
plt.title('Left ankle Angle in degrees ')
plt.xlabel('time(s)')
plt.ylabel('Angle(degrees)')
# plt.legend(['mediapipe','mocap'])
plt.ylim(0,180)
plt.show()
# print('RMSE of left knee Angle:',RMSE(mocapLKa,mpipeLKa,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))

print(mpipeRAa)
print(mpipeLAa)

In [ ]:
def shoulder_frame(S,E,W):
    """"
        Find the vectors describing the local x,y,z axes of shoulder
    Parameters
    ----------
    S : shoulder point
    E : elbow point
    W : wrist point
        
    Returns
    -------
    numpy.ndarray
        An array of orthonormal vectors, where each column represents a vector.
    """
    S,E,W=np.array(S),np.array(E),np.array(W)
    upperarm=E-S
    lowerarm=W-E

    y=upperarm
    x=np.cross(upperarm,lowerarm)
    z=np.cross(x,y)
    return np.array([x,y,z]).T

def tr_frame(LS,RS,TR):
    """"
        Find the vectors describing the local x,y,z axes of trunk
    Parameters
    ----------
    LS : left shoulder point
    RS : right shoulder point
    TR : trunk/chest point
    
    Returns
    -------
    numpy.ndarray
        An array of orthonormal vectors, where each column represents a vector.
    """

    LS,RS,TR=np.array(LS),np.array(RS),np.array(TR)

    v1=LS-RS
    v2=TR-RS

    basis=gram_schmidt([v1,v2])
    try:
        v1,v2=basis[0],basis[1]
    except:
        return np.nan
    v3=np.cross(v1,v2)

    return np.array([v1,v2,v3]).T

def gram_schmidt(vectors):
    """
        Orthonormalize a set of vectors using the Gram-Schmidt process.

    Parameters
    ----------
    vectors : list of numpy.ndarray
        A list of vectors to be orthonormalized.

    Returns
    -------
    numpy.ndarray
        An array of orthonormal vectors, where each row represents a vector.
    """
    basis = []
    for v in vectors:
        w = v - sum(np.dot(v,b)*b for b in basis)
        if (w > 1e-10).any():
            basis.append(w/np.linalg.norm(w))
    return np.array(basis)


def rotation_matrix(frame1, frame2):
    """
    Calculate the rotation matrix that transforms frame1 to frame2.
    :param frame1: 3x3 numpy array representing the first coordinate frame
    :param frame2: 3x3 numpy array representing the second coordinate frame
    :return: 3x3 numpy array representing the rotation matrix
    """
    R = np.dot(frame2, np.linalg.inv(frame1))
    return R


In [ ]:
# callibrating shoulders and trunk

RHC_mpipe=shoulder_frame([df_mpipe['RH_x'][mpipe_cf],df_mpipe['RH_y'][mpipe_cf],df_mpipe['RH_z'][mpipe_cf]],
                         [df_mpipe['RK_x'][mpipe_cf],df_mpipe['RK_y'][mpipe_cf],df_mpipe['RK_z'][mpipe_cf]],
                         [df_mpipe['RA_x'][mpipe_cf],df_mpipe['RA_y'][mpipe_cf],df_mpipe['RA_z'][mpipe_cf]])

LHC_mpipe=shoulder_frame([df_mpipe['LH_x'][mpipe_cf],df_mpipe['LH_y'][mpipe_cf],df_mpipe['LH_z'][mpipe_cf]],
                         [df_mpipe['LK_x'][mpipe_cf],df_mpipe['LK_y'][mpipe_cf],df_mpipe['LK_z'][mpipe_cf]],
                         [df_mpipe['LA_x'][mpipe_cf],df_mpipe['LA_y'][mpipe_cf],df_mpipe['LA_z'][mpipe_cf]])

TRC_mpipe=tr_frame([df_mpipe['LH_x'][mpipe_cf],df_mpipe['LH_y'][mpipe_cf],df_mpipe['LH_z'][mpipe_cf]],
                   [df_mpipe['RH_x'][mpipe_cf],df_mpipe['RH_y'][mpipe_cf],df_mpipe['RH_z'][mpipe_cf]],
                   [df_mpipe['TR_x'][mpipe_cf],df_mpipe['TR_y'][mpipe_cf],df_mpipe['TR_z'][mpipe_cf]])

RHC_mocap=shoulder_frame([df_mocap['RH_x'][mocap_cf],df_mocap['RH_y'][mocap_cf],df_mocap['RH_z'][mocap_cf]],
                         [df_mocap['RK_x'][mocap_cf],df_mocap['RK_y'][mocap_cf],df_mocap['RK_z'][mocap_cf]],
                         [df_mocap['RA_x'][mocap_cf],df_mocap['RA_y'][mocap_cf],df_mocap['RA_z'][mocap_cf]])

LHC_mocap=shoulder_frame([df_mocap['LH_x'][mocap_cf],df_mocap['LH_y'][mocap_cf],df_mocap['LH_z'][mocap_cf]],
                         [df_mocap['LK_x'][mocap_cf],df_mocap['LK_y'][mocap_cf],df_mocap['LK_z'][mocap_cf]],
                         [df_mocap['LA_x'][mocap_cf],df_mocap['LA_y'][mocap_cf],df_mocap['LA_z'][mocap_cf]])

TRC_mocap=tr_frame([df_mocap['LH_x'][mocap_cf],df_mocap['LH_y'][mocap_cf],df_mocap['LH_z'][mocap_cf]],
                   [df_mocap['RH_x'][mocap_cf],df_mocap['RH_y'][mocap_cf],df_mocap['RH_z'][mocap_cf]],
                   [df_mocap['TR_x'][mocap_cf],df_mocap['TR_y'][mocap_cf],df_mocap['TR_z'][mocap_cf]])

In [ ]:
# calculating the euler angles

# mpipe values
mpipeRHFE,mpipeRHAA,mpipeRHIE=[],[],[]
mpipeLHFE,mpipeLHAA,mpipeLHIE=[],[],[]
mpipeTRFB,mpipeTRLR,mpipeTRRO=[],[],[]

for i in range(len(df_mpipe['epoch_time'].tolist())):

    # finding the ith shoulder/trunk frames
    mpipeRHF=shoulder_frame([df_mpipe['RH_x'][i],df_mpipe['RH_y'][i],df_mpipe['RH_z'][i]],
                            [df_mpipe['RK_x'][i],df_mpipe['RK_y'][i],df_mpipe['RK_z'][i]],
                            [df_mpipe['RA_x'][i],df_mpipe['RA_y'][i],df_mpipe['RA_z'][i]])
   
    mpipeLHF=shoulder_frame([df_mpipe['LH_x'][i],df_mpipe['LH_y'][i],df_mpipe['LH_z'][i]],
                            [df_mpipe['LK_x'][i],df_mpipe['LK_y'][i],df_mpipe['LK_z'][i]],
                            [df_mpipe['LA_x'][i],df_mpipe['LA_y'][i],df_mpipe['LA_z'][i]])
   
    mpipeTRF=tr_frame([df_mpipe['LH_x'][i],df_mpipe['LH_y'][i],df_mpipe['LH_z'][i]],
                      [df_mpipe['RH_x'][i],df_mpipe['RH_y'][i],df_mpipe['RH_z'][i]],  
                      [df_mpipe['TR_x'][i],df_mpipe['TR_y'][i],df_mpipe['TR_z'][i]])
   
    # finding the rotation matrix from callibration trunk frame to ith trunk frame
    mpipeTR_rotmat=rotation_matrix(TRC_mpipe,mpipeTRF)

    # rotating shoulder callibration frame by same the rotation matrix
    mpipe_rotated_RH_Cframe=mpipeTR_rotmat@RHC_mpipe
    mpipe_rotated_LH_Cframe=mpipeTR_rotmat@LHC_mpipe

    # now finding rotation matrix from rotated shoulder callibration frame to ith shoulder frame
    mpipeRH_rotmat=rotation_matrix(mpipe_rotated_RH_Cframe,mpipeRHF)
    mpipeLH_rotmat=rotation_matrix(mpipe_rotated_LH_Cframe,mpipeLHF)

    # finding euler angles from rotation matrices
    RHeuler=rotation_angles(mpipeRH_rotmat)
    LHeuler=rotation_angles(mpipeLH_rotmat)
    TReuler=rotation_angles(mpipeTR_rotmat)

    # appending to angle lists
    mpipeRHFE.append(RHeuler[0])
    mpipeRHAA.append(RHeuler[1])
    mpipeRHIE.append(RHeuler[2])

    mpipeLHFE.append(LHeuler[0])
    mpipeLHAA.append(LHeuler[1])
    mpipeLHIE.append(LHeuler[2])

    mpipeTRFB.append(TReuler[0])
    mpipeTRLR.append(TReuler[1])
    mpipeTRRO.append(TReuler[2])

# mocap values
mocapRHFE,mocapRHAA,mocapRHIE=[],[],[]
mocapLHFE,mocapLHAA,mocapLHIE=[],[],[]
mocapTRFB,mocapTRLR,mocapTRRO=[],[],[]


for i in range(len(df_mocap['epoch_time'].tolist())):

    # finding the ith shoulder/trunk frames
    mocapRHF=shoulder_frame([df_mocap['RH_x'][i],df_mocap['RH_y'][i],df_mocap['RH_z'][i]],
                            [df_mocap['RK_x'][i],df_mocap['RK_y'][i],df_mocap['RK_z'][i]],
                            [df_mocap['RA_x'][i],df_mocap['RA_y'][i],df_mocap['RA_z'][i]])
   
    mocapLHF=shoulder_frame([df_mocap['LH_x'][i],df_mocap['LH_y'][i],df_mocap['LH_z'][i]],
                            [df_mocap['LK_x'][i],df_mocap['LK_y'][i],df_mocap['LK_z'][i]],
                            [df_mocap['LA_x'][i],df_mocap['LA_y'][i],df_mocap['LA_z'][i]])
   
    mocapTRF=tr_frame([df_mocap['LH_x'][i],df_mocap['LH_y'][i],df_mocap['LH_z'][i]],
                      [df_mocap['RH_x'][i],df_mocap['RH_y'][i],df_mocap['RH_z'][i]],  
                      [df_mocap['TR_x'][i],df_mocap['TR_y'][i],df_mocap['TR_z'][i]])
   
    # finding the rotation matrix from callibration trunk frame to ith trunk frame
    mocapTR_rotmat=rotation_matrix(TRC_mocap,mocapTRF)

    # rotating shoulder callibration frame by same the rotation matrix
    mocap_rotated_RH_Cframe=mocapTR_rotmat@RHC_mocap
    mocap_rotated_LH_Cframe=mocapTR_rotmat@LHC_mocap

    # now finding rotation matrix from rotated shoulder callibration frame to ith shoulder frame
    mocapRH_rotmat=rotation_matrix(mocap_rotated_RH_Cframe,mocapRHF)
    mocapLH_rotmat=rotation_matrix(mocap_rotated_LH_Cframe,mocapLHF)

    # finding euler angles from rotation matrices
    RHeuler=rotation_angles(mocapRH_rotmat)
    LHeuler=rotation_angles(mocapLH_rotmat)
    TReuler=rotation_angles(mocapTR_rotmat)

    # appending to angle lists
    mocapRHFE.append(RHeuler[0])
    mocapRHAA.append(RHeuler[1])
    mocapRHIE.append(RHeuler[2])

    mocapLHFE.append(LHeuler[0])
    mocapLHAA.append(LHeuler[1])
    mocapLHIE.append(LHeuler[2])
   
    mocapTRFB.append(TReuler[0])
    mocapTRLR.append(TReuler[1])
    mocapTRRO.append(TReuler[2])

In [ ]:
# Plotting the shoulder angles
pvt(mpipeRHFE,df_mpipe['epoch_time'])
pvt(mocapRHFE,df_mocap['epoch_time'])
plt.legend(['mediapipe','mocap'])
plt.title('right shoudler flexion')
plt.ylim(-90,90)
plt.show()


print('RMHE of flexion/extension: ',RMHE(mocapRHFE,mpipeRHFE,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))


pvt(mpipeRHAA,df_mpipe['epoch_time'])
pvt(mocapRHAA,df_mocap['epoch_time'])
plt.legend(['mediapipe','mocap'])
plt.title('right shoulder abduction')
plt.ylim(-90,90)
plt.show()


print('RMHE of abduct/adduct: ',RMHE(mocapRHAA,mpipeRHAA,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))


pvt(mpipeRHIE,df_mpipe['epoch_time'].tolist())
pvt(mocapRHIE,df_mocap['epoch_time'].tolist())
plt.legend(['mediapipe','mocap'])
plt.title('right shoulder internal/external rotation')
plt.ylim(-90,90)
plt.show()


print('RMHE of internal/external rotation: ',RMHE(mocapRHIE,mpipeRHIE,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))


pvt(mpipeLHFE,df_mpipe['epoch_time'])
pvt(mocapLHFE,df_mocap['epoch_time'])
plt.legend(['mediapipe','mocap'])
plt.title('left shoulder flexion')
plt.ylim(-90,90)
plt.show()


print('RMHE of flexion/extension: ',RMHE(mocapLHFE,mpipeLHFE,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))


pvt(mpipeLHAA,df_mpipe['epoch_time'])
pvt(mocapLHAA,df_mocap['epoch_time'])
plt.legend(['mediapipe','mocap'])
plt.title('left shoulder abduction')
plt.ylim(-90,90)
plt.show()


print('RMHE of abduct/adduct: ',RMHE(mocapLHAA,mpipeLHAA,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))


pvt(mpipeLHIE,df_mpipe['epoch_time'].tolist())
pvt(mocapLHIE,df_mocap['epoch_time'].tolist())
plt.legend(['mediapipe','mocap'])
plt.title('left shoulder internal/external rotation')
plt.ylim(-90,90)
plt.show()


print('RMHE of internal/external rotation: ',RMHE(mocapLHIE,mpipeLHIE,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))